Import Requirements

In [147]:
import pyodbc
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime, timedelta

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from transform import remove_html, maintain_alpha, remove_single, remove_morespace, remove_enumerate, clean_text, remove_insideparentheses, remove_standalonesymbols, stopwords_remover

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eats\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Required Field

In [148]:
'''
Field yang required*

Applicant
- Dob
- ExpectedSalary
- IsAbleToWorkRemote
- CityID
- ProvinceID
- DriverLicenseType
- Gender
- IsUsingGlasses
- Height
- MaritalStatus
- Nationality
- Strengthness
- Weaknesses
- TypeOfVehicle
- Weight
- CurrentAddressCityID
- CurrentAddressProvinceID

ApplicantEducation
- DateEnd
- DateStart
- Score
- EducationLevelID
- MajorID

ApplicantExperience
- DateTo
- DateFrom
- Industry
- CompanyName
- JobDescription
- Position
- Salary
- 
'''

'\nField yang required*\n\nApplicant\n- Dob\n- ExpectedSalary\n- IsAbleToWorkRemote\n- CityID\n- ProvinceID\n- DriverLicenseType\n- Gender\n- IsUsingGlasses\n- Height\n- MaritalStatus\n- Nationality\n- Strengthness\n- Weaknesses\n- TypeOfVehicle\n- Weight\n- CurrentAddressCityID\n- CurrentAddressProvinceID\n\nApplicantEducation\n- DateEnd\n- DateStart\n- Score\n- EducationLevelID\n- MajorID\n\nApplicantExperience\n- DateTo\n- DateFrom\n- Industry\n- CompanyName\n- JobDescription\n- Position\n- Salary\n- \n'

##### Code

In [149]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

Connection

In [150]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [151]:
tables = engine.table_names()

C:\Users\eats\AppData\Local\Temp\ipykernel_8328\863585613.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


In [152]:
df_applicant = pd.DataFrame(engine.execute('SELECT ApplicantID, DiseaseHistory, Dob, ExpectedSalary, IsAbleToWorkRemote, CityID, ProvinceID, DriverLicenseType, Gender, IsUsingGlasses, Height, MaritalStatus, Nationality, Strengthness, Weaknesses, TypeOfVehicle FROM Applicant'))
df_applicant_education = pd.DataFrame(engine.execute('SELECT ApplicantEducationID, ApplicantID, DateStart, DateEnd, EducationInstituteName, Score, EducationLevelID, MajorID, Degree FROM ApplicantEducation'))
df_applicant_experience = pd.DataFrame(engine.execute('SELECT ApplicantExperienceID, ApplicantID, DateFrom, DateTo, Industry, CompanyName, JobDescription, Position, Salary FROM ApplicantExperience'))
df_applicant_document = pd.DataFrame(engine.execute('SELECT ApplicantID, DocumentName FROM ApplicantDocument'))
df_applicant_certificate = pd.DataFrame(engine.execute('SELECT ApplicantID, Description FROM ApplicantCertificate'))

df_pipeline = pd.DataFrame(engine.execute('SELECT PipelineID, ApplicantID, JobID, StageID FROM Pipeline'))
df_stage = pd.DataFrame(engine.execute('SELECT StageID, Label FROM Stage'))

df_job = pd.DataFrame(engine.execute('SELECT * FROM Job'))
df_function_position = pd.DataFrame(engine.execute('SELECT FunctionPositionID, FunctionPositionName FROM FunctionPosition'))
df_department = pd.DataFrame(engine.execute('SELECT DepartmentID, Name AS DepartmentName FROM Department'))
df_city = pd.DataFrame(engine.execute('SELECT CityID, Name AS CityName FROM City'))
df_province = pd.DataFrame(engine.execute('SELECT ProvinceID, Name AS ProvinceName FROM Province'))
df_major = pd.DataFrame(engine.execute('SELECT MajorID, MajorName FROM Major'))
df_education_level = pd.DataFrame(engine.execute('SELECT EducationLevelID, EducationLevelName FROM EducationLevel'))
df_company = pd.DataFrame(engine.execute('SELECT CompanyID, Name AS CompanyName FROM Company'))

# df_job = df_job[['JobID', 'Description', 'EducationLevelID', 'FunctionPositionID', 'DepartmentID', 'JobTitle', 'Requirement', 'CityID', 'ProvinceID', 'MajorID']]

In [7]:
df_job.head(3)

,JobID,Description,EducationLevelID,FunctionPositionID,DepartmentID,JobTitle,Requirement,CityID,ProvinceID,MajorID
0,1,Under Asst. Front Office Manager at Gunawangsa...,4,1,1,NIGHT AUDIT,"<p>\r\n\r\n</p><div style=""language:en-US;marg...",1,1,1
1,2,BECOMING BOOK KEEPER FOR GUNAWANGSA HOTEL MERR...,2,2,2,BOOK KEEPER,<p>Bachelor Degree of Accounting.<br>With 2 ye...,1,1,2
2,3,"<ul style=""padding: 0px 40px; color: rgb(51, 5...",2,3,8,IT ANDROID PROGRAMMER,"<p></p><p><ul></ul><span style=""font-size:12.0...",1,1,4


In [8]:
df_job.Description.fillna('', inplace=True)
df_job.Requirement.fillna('', inplace=True)

In [9]:
df_merged = pd.merge(df_job, df_education_level, on=['EducationLevelID'])
df_merged = pd.merge(df_merged, df_city, on=['CityID'])
df_merged = pd.merge(df_merged, df_province, on=['ProvinceID'])
df_merged = pd.merge(df_merged, df_function_position, on=['FunctionPositionID'])
df_merged = pd.merge(df_merged, df_department, on=['DepartmentID'])
df_merged = pd.merge(df_merged, df_major, on=['MajorID'])

df_merged.drop(columns=['EducationLevelID', 'CityID', 'ProvinceID', 'FunctionPositionID', 'DepartmentID', 'MajorID'], inplace=True)

In [10]:
df_merged.drop(
    index=df_merged.index[df_merged.JobTitle.map(str.lower).str.contains('test|123')].values, inplace=True
)

In [11]:
df_merged.head(2)

,JobID,Description,JobTitle,Requirement,EducationLevelName,CityName,ProvinceName,FunctionPositionName,DepartmentName,MajorName
0,1,Under Asst. Front Office Manager at Gunawangsa...,NIGHT AUDIT,"<p>\r\n\r\n</p><div style=""language:en-US;marg...",D3,SURABAYA,JAWA TIMUR,NIGHT AUDIT,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN
2,48,<p>Becoming Executive Housekeeping For Gunawan...,EXECUTIVE HOUSEKEEPING,<p>Male<br>Experience in hospitality industry ...,D3,SURABAYA,JAWA TIMUR,EXECUTIVE HOUSEKEEPING,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN


In [12]:
df_merged.Description = df_merged.Description.map(clean_text)

In [13]:
df_merged.Requirement = df_merged.Requirement.map(remove_html).map(remove_enumerate).map(maintain_alpha).map(remove_single).map(remove_morespace).map(str.strip).map(str.lower)

In [14]:
df_merged.JobTitle = df_merged.JobTitle.map(str.lower).map(remove_insideparentheses).map(remove_standalonesymbols).map(remove_morespace)

In [15]:
df_merged.EducationLevelName = df_merged.EducationLevelName.replace('None', '').map(str.lower)

In [16]:
df_merged.CityName = df_merged.CityName.map(str.lower)
df_merged.ProvinceName = df_merged.ProvinceName.map(str.lower)

In [17]:
df_merged.FunctionPositionName = df_merged.FunctionPositionName.map(remove_standalonesymbols).apply(lambda x: re.sub('[\(\)0-9]', '', x)).map(remove_morespace).map(str.strip).map(str.lower)

In [18]:
df_merged.head(2)

,JobID,Description,JobTitle,Requirement,EducationLevelName,CityName,ProvinceName,FunctionPositionName,DepartmentName,MajorName
0,1,under asst front office manager at gunawangsa ...,night audit,with 1 year minimum experiences in the same po...,d3,surabaya,jawa timur,night audit,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN
2,48,becoming executive housekeeping for gunawangsa...,executive housekeeping,male experience in hospitality industry and ex...,d3,surabaya,jawa timur,executive housekeeping,FRONT OFFICE GUNAWANGSA HOTEL MERR,PERHOTELAN


In [19]:
df_merged = df_merged[['JobID', 'JobTitle', 'FunctionPositionName', 'EducationLevelName', 'CityName', 'ProvinceName', 'Description', 'Requirement']]

In [20]:
df_merged

,JobID,JobTitle,FunctionPositionName,EducationLevelName,CityName,ProvinceName,Description,Requirement
0,1,night audit,night audit,d3,surabaya,jawa timur,under asst front office manager at gunawangsa ...,with 1 year minimum experiences in the same po...
2,48,executive housekeeping,executive housekeeping,d3,surabaya,jawa timur,becoming executive housekeeping for gunawangsa...,male experience in hospitality industry and ex...
3,49,executive housekeeping,executive housekeeping,d3,surabaya,jawa timur,becoming executive housekeeping for gunawangsa...,male experience in hospitality industry and ex...
4,57,food and beverages manager,food and beverages manager,d3,surabaya,jawa timur,become asst food beverages manager at gunawang...,male experience in hospitality industry at ass...
5,1576,cashier f&b,receptionist,smk,surabaya,jawa timur,become receptionis and cashier,female graduate from hospitality vocational hi...
...,...,...,...,...,...,...,...,...
1002,1661,translator staff,administrasi,s1,surabaya,jawa timur,membuat teks terjemahan yang akurat menulis da...,usia maksimal 25 thn pendidikan minimal d3 lin...
1003,1334,english teacher,teacher,s1,jakarta,dki jakarta,membuat laporan mengajar dan administrasi ke p...,pengalaman 3 tahun sebagai teacher di preschoo...
1004,1413,assistant teacher,teacher,s1,jakarta,dki jakarta,good teamwork to communicate with colleagues a...,english communication both oral and written is...
1005,1415,teacher,teacher,s1,jakarta,dki jakarta,good teamwork to communicate with colleagues a...,english communication both oral and written is...


In [21]:
df_merged = df_merged.set_index(['JobID'])

In [22]:
df_train = pd.DataFrame([], index=df_merged.index)

In [29]:
df_train['Text'] = df_merged.JobTitle.str.cat(
    df_merged.FunctionPositionName.str.cat(
        df_merged.EducationLevelName.str.cat(
            df_merged.CityName.str.cat(
                df_merged.ProvinceName.str.cat(
                    df_merged.Description.str.cat(
                        df_merged.Requirement
                    , sep=' ')
                , sep=' ')
            , sep=' ')
        , sep=' ')
    , sep=' ')
, sep=' ')

In [24]:
sastrawi_stopwords = StopWordRemoverFactory().get_stop_words()
nltk_stopwords =  stopwords.words('indonesian')

In [134]:
stopwords_in = list(set(sastrawi_stopwords + nltk_stopwords))

In [115]:
df_train.Text = df_train.Text.apply(lambda x: re.sub('\s+', ' ', x)).apply(lambda x: re.sub('(' + ' | '.join(stopwords_in) + ')', ' ', x)).map(remove_morespace)

In [136]:
from eris import ErisRecommender

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [137]:
encoder = TfidfVectorizer()
bank = encoder.fit_transform(df_train.Text)
# code = encoder.transform(job_train.Text)
# dist = cosine_similarity(code, bank)[0]*100

In [138]:
eris = ErisRecommender(df_merged, df_train, 'Text')
eris.fit()
eris.recommend('programmer')

Index yang ada 1222


,JobTitle,Description,Requirement,Similarity
JobID,,,,
1222,it staff,menerima memprioritaskan dan menyelesaikan per...,usia maksimal 35 tahun pendidikan minimal s1 t...,100
102,it support,menerima memprioritaskan dan menyelesaikan per...,usia maksimal 35 tahun pendidikan minimal s1 s...,99
1177,it staff,menerima memprioritaskan dan menyelesaikan per...,usia maksimal 35 tahun pendidikan minimal s1 t...,99
1193,it staff,menerima memprioritaskan dan menyelesaikan per...,usia maksimal 35 tahun pendidikan minimal s1 t...,98
3050,it support,bertanggungjawab pada kesiapan dan ketersediaa...,usia maksimal 35 tahun pendidikan minimal d3 i...,26
...,...,...,...,...
2875,cleaning service,membersihkan area kantor merawat alat kebersih...,laki laki max 30 tahun memiliki pengalaman dib...,0
2867,cleaning service,membersihkan area kantor merawat alat kebersih...,laki laki max 30 tahun memiliki pengalaman dib...,0
41,staff research development,melakukan analisa,laki laki atau perempuan usia maksimal 35 tahu...,0


In [140]:
df_merged

,JobTitle,FunctionPositionName,EducationLevelName,CityName,ProvinceName,Description,Requirement
JobID,,,,,,,
1,night audit,night audit,d3,surabaya,jawa timur,under asst front office manager at gunawangsa ...,with 1 year minimum experiences in the same po...
48,executive housekeeping,executive housekeeping,d3,surabaya,jawa timur,becoming executive housekeeping for gunawangsa...,male experience in hospitality industry and ex...
49,executive housekeeping,executive housekeeping,d3,surabaya,jawa timur,becoming executive housekeeping for gunawangsa...,male experience in hospitality industry and ex...
57,food and beverages manager,food and beverages manager,d3,surabaya,jawa timur,become asst food beverages manager at gunawang...,male experience in hospitality industry at ass...
1576,cashier f&b,receptionist,smk,surabaya,jawa timur,become receptionis and cashier,female graduate from hospitality vocational hi...
...,...,...,...,...,...,...,...
1661,translator staff,administrasi,s1,surabaya,jawa timur,membuat teks terjemahan yang akurat menulis da...,usia maksimal 25 thn pendidikan minimal d3 lin...
1334,english teacher,teacher,s1,jakarta,dki jakarta,membuat laporan mengajar dan administrasi ke p...,pengalaman 3 tahun sebagai teacher di preschoo...
1413,assistant teacher,teacher,s1,jakarta,dki jakarta,good teamwork to communicate with colleagues a...,english communication both oral and written is...


In [157]:
df_job[df_job.JobID == 1661]

,JobID,Description,EducationLevelID,FunctionPositionID,JobStatus,JobTitle,PostedDate,Requirement,SalaryMax,SalaryMin,UserId,HiredQuota,ClosedDate,DepartmentID,CityID,CompanyID,ProvinceID,SkipTest,MajorID,DbName,UsiaMax,UsingGlasses,IQMin,IsSpecificEducationLevel,DriverLicenseType,IsSpesificDriverLicense,Gender,IsSpecificAge,IsSpecificGender,IsSpecificIQ,IsSpecificMarital,MaritalStatus,PublishDate,AdsStatisticID
634,1661,"<ul type=""disc""><li class=""MsoNormal"" style=""c...",2,36,Close,TRANSLATOR STAFF,2021-05-10 00:00:00.0000000,"<p>\r\n\r\n</p><br>\r\n\r\n<ul type=""disc""><li...",0.0,0.0,aab981ee-3ee0-464e-8cd4-8a033918050a,1,2021-06-17 13:42:35.4541978,16,1,1,1,True,20,WarnaWarni,0.0,False,0,False,None,False,,False,False,False,False,None,0001-01-01 00:00:00.0000000,None


In [164]:
df_job.Requirement[df_job.JobID == 1661].map(clean_text).values

array(['usia maksimal 25 thn pendidikan minimal d3 linguistik bahasa memiliki ketrampilan bahasa inggris dengan baik pengalaman sebagai penerjemah lebih disukai'],
      dtype=object)

In [158]:
df_education_level

,EducationLevelID,EducationLevelName
0,4,D3
1,3,D4
2,9,None
3,2,S1
4,1,S2
5,10,S3
6,8,SD
7,5,SMA
8,6,SMK
9,7,SMP


In [146]:
df_job

,JobID,Description,EducationLevelID,FunctionPositionID,DepartmentID,JobTitle,Requirement,CityID,ProvinceID,MajorID
0,1,Under Asst. Front Office Manager at Gunawangsa...,4,1,1,NIGHT AUDIT,"<p>\r\n\r\n</p><div style=""language:en-US;marg...",1,1,1
1,2,BECOMING BOOK KEEPER FOR GUNAWANGSA HOTEL MERR...,2,2,2,BOOK KEEPER,<p>Bachelor Degree of Accounting.<br>With 2 ye...,1,1,2
2,3,"<ul style=""padding: 0px 40px; color: rgb(51, 5...",2,3,8,IT ANDROID PROGRAMMER,"<p></p><p><ul></ul><span style=""font-size:12.0...",1,1,4
3,4,"<div style=""text-align: justify;""><ul><li styl...",2,4,8,SEKRETARIS LEGAL,"<div style=""color: rgb(51, 51, 51); font-famil...",1,1,6
4,5,"<ul style=""padding: 0px 40px; color: rgb(51, 5...",4,5,8,SEKRETARIS DIREKSI,"<ul style=""padding: 0px 40px; color: rgb(51, 5...",1,1,7
...,...,...,...,...,...,...,...,...,...,...
1003,3090,"<ul style=""margin-top:0in"" type=""square"">\r\n ...",9,46,13,GENERAL MANAGER HOTEL,"<ul type=""disc"">\r\n <li class=""MsoNormal"" sty...",1,1,7
1004,3091,,9,6,10,MARKETING STAFF,,1,1,7
1005,3092,"<ul style=""color: rgb(33, 37, 41);""><li>Mengua...",2,39,49,MANAGER FINANCE & ACCOUNTING,"<ul style=""color: rgb(33, 37, 41);""><li>Usia m...",2,2,2
1006,3093,"<p class=""MsoNormal"" style=""margin: 0in 0in 7....",9,26,13,GENERAL AFFAIR (GA),"<ul type=""disc"" style=""color: rgb(51, 51, 51);...",1,1,15


In [154]:
df_job[df_job.JobID == 48]

,JobID,Description,EducationLevelID,FunctionPositionID,JobStatus,JobTitle,PostedDate,Requirement,SalaryMax,SalaryMin,UserId,HiredQuota,ClosedDate,DepartmentID,CityID,CompanyID,ProvinceID,SkipTest,MajorID,DbName,UsiaMax,UsingGlasses,IQMin,IsSpecificEducationLevel,DriverLicenseType,IsSpesificDriverLicense,Gender,IsSpecificAge,IsSpecificGender,IsSpecificIQ,IsSpecificMarital,MaritalStatus,PublishDate,AdsStatisticID
39,48,<p>Becoming Executive Housekeeping For Gunawan...,4,38,Close,EXECUTIVE HOUSEKEEPING,2018-07-03 00:00:00.0000000,<p>Male<br>Experience in hospitality industry ...,6000000.0,3587000.0,124839f6-8e58-48e8-8959-968da693a5b1,1,2018-09-12 11:35:14.7251826,1,1,7,1,False,1,WarnaWarni,0.0,False,0,False,None,False,,False,False,False,False,None,0001-01-01 00:00:00.0000000,None


In [156]:
df_job.Requirement[df_job.JobID == 48].values

array(['<p>Male<br>Experience in hospitality industry and Executive Housekeeping or Asst. Executive Housekeeping Min 1 year<br></p><p>Has Leader Ability<br>Has good personality &amp; attitude<br>Willing to become a key player in a dynamic team and able to work as a team.</p>'],
      dtype=object)

Applicant

In [139]:
df_applicant

,ApplicantID,DiseaseHistory,Dob,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle
0,1,,2018-04-10 09:55:35.1320927,0.0,False,NaN,NaN,,,False,0,,,,,
1,2,-,2018-04-10 00:00:00.0000000,0.0,False,1.0,1.0,A,Male,False,0,None,Indonesia,strength,weak,Mobil
2,3,none,1994-01-06 00:00:00.0000000,4000000.0,False,1.0,1.0,C,Male,False,178,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor
3,4,-,1995-05-25 00:00:00.0000000,4000000.0,True,1.0,1.0,C,Male,False,180,Single,Indonesia,cepat mengerti,kurang tegas,Sepeda Motor
4,5,,2018-04-11 07:22:05.6383519,0.0,False,NaN,NaN,,,False,0,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34986,44927,,None,0.0,False,NaN,NaN,,,None,0,,,,,
34987,44928,,None,0.0,False,NaN,NaN,,,None,0,,,,,
34988,44929,,None,0.0,False,NaN,NaN,,,None,0,,,,,
34989,44930,,None,0.0,False,NaN,NaN,,,None,0,,,,,


##### Tambah stopwords jika akurasi kurang

In [118]:
text = ''

for txt in df_train.Text.values:
    text += txt

In [120]:
tes = text.split(' ')

In [124]:
'memiliki' in stopwords_in

False

In [ ]:
stopwords_in = list(set(sastrawi_stopwords + nltk_stopwords)) + ['memiliki', 'minimal', ]

In [133]:
pd.Series(tes)[~pd.Series(tes).str.contains('memiliki|minimal|and|menguasai|')].value_counts().head(20)

s1            940
jakarta       930
perusahaan    929
pengalaman    870
usia          745
pendidikan    719
surabaya      681
timur         626
jawa          606
maksimal      605
kerja         519
1             478
in            440
menguasai     435
35            431
d3            424
laporan       416
hukum         413
legal         403
dki           397
dtype: int64